In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras_tuner import HyperModel, Hyperband

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [5]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = tf.keras.models.Sequential()
        model.add(Conv2D(
            filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('conv_1_kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=self.input_shape
        ))
        model.add(MaxPooling2D(2, 2))
        model.add(Conv2D(
            filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=32),
            kernel_size=hp.Choice('conv_2_kernel_size', values=[3, 5]),
            activation='relu'
        ))
        model.add(MaxPooling2D(2, 2))
        model.add(Conv2D(
            filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=32),
            kernel_size=hp.Choice('conv_3_kernel_size', values=[3, 5]),
            activation='relu'
        ))
        model.add(MaxPooling2D(2, 2))
        model.add(Flatten())
        model.add(Dense(
            hp.Int('dense_units', min_value=128, max_value=512, step=32),
            activation='relu'
        ))
        model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
        model.add(Dense(1, activation='sigmoid'))
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model

In [6]:
number_of_epochs = 40

In [7]:
hypermodel = CNNHyperModel(input_shape=(150, 150, 3))

In [8]:

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='hyperband',
    project_name='mushroom_classification'
)

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
tuner.search(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

Trial 30 Complete [00h 02m 09s]
val_accuracy: 0.4285714328289032

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 21m 13s


In [10]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

In [11]:
# Train the best model
model.fit(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

Epoch 1/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.4389 - loss: 2.2926 - val_accuracy: 0.4286 - val_loss: 0.8382
Epoch 2/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6140 - loss: 0.9939 - val_accuracy: 0.4286 - val_loss: 0.7162
Epoch 3/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5093 - loss: 0.6996 - val_accuracy: 0.4286 - val_loss: 0.7175
Epoch 4/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5673 - loss: 0.6827 - val_accuracy: 0.4286 - val_loss: 0.7171
Epoch 5/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5968 - loss: 0.6849 - val_accuracy: 0.4286 - val_loss: 0.7132
Epoch 6/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5751 - loss: 0.6859 - val_accuracy: 0.4286 - val_loss: 0.7387
Epoch 7/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5833 - loss: 0.6909 - val_accuracy: 0.4286 - val_loss: 0.7435
Epoch 8/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5454 - loss: 0.7034 - val_accuracy: 0.4286 - val_loss: 0.7234
Epoch 9/

In [12]:
loss, accuracy = model.evaluate(val_generator)
print(f"Final validation accuracy: {accuracy*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4286 - loss: 0.7699
Final validation accuracy: 42.86%


In [13]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_test_{number_of_epochs}.keras')